# Telecom Churn Case Study

### Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA
from sklearn.decomposition import IncrementalPCA

from sklearn.ensemble import RandomForestClassifier,  GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error


from sklearn.metrics import r2_score
from sklearn.metrics import precision_recall_curve

from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics

import statsmodels.api as sm

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import roc_curve, auc

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
#read
data = pd.read_csv('telecom_churn_data.csv')

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
#spread of data
data.describe()

In [ ]:
#data types
data.dtypes

In [ ]:
#check unique values
#if output equals 99999 no duplicates
data['mobile_number'].nunique()

### Derive Features

In [ ]:
data['avg_rec_amt'] = (data['total_rech_amt_6'] + data['total_rech_amt_7'])/2
data['avg_rec_amt'].head()

In [ ]:
df = data.copy()
df.head(3)

In [ ]:
#drop columns
df = df.drop(columns=['total_rech_amt_6', 'total_rech_amt_7'], axis=1)

In [ ]:
df.shape

### Filter out high value customers

In [ ]:
#define X
X = df['avg_rec_amt'].quantile(0.70)
print('X:',X)

In [ ]:
#filter high value
df_high_value = df[(df['avg_rec_amt'] > X)]

In [ ]:
df_high_value.shape

#### Define Total usage

In [ ]:
#define total_usage
df_high_value['total_usage'] = (df_high_value['total_ic_mou_9'] + df_high_value['total_og_mou_9'] + 
                               df_high_value['vol_2g_mb_9'] + df_high_value['vol_3g_mb_9'])
df_high_value['total_usage'].head()

In [ ]:
df_high_value.shape

#### Label 1 if churn else 0

In [ ]:
#where total_usage == 0, label 1, else 0
df_high_value['churn'] = np.where(df_high_value['total_usage'] == 0, 1, 0)
df_high_value['churn'].head()

In [ ]:
df_high_value.shape

#### Churn Rate

In [ ]:
churn_rate = round(df_high_value['churn'].sum()/len(df_high_value.index), 4)*100
print('Churn Rate:', churn_rate,'%')

#### Filter out columns for month 9

In [ ]:
drop_9 = [feature for feature in df_high_value.columns if '_9' in feature]
print(len(drop_9))

In [ ]:
df_high_value = df_high_value.drop(drop_9, axis=1)

In [ ]:
df_high_value.shape

In [ ]:
df_high_value.head(3)

#### Filter out columns with more than 60% null values

In [ ]:
#filter null cols > 60%
null_cols = [feature for feature in df_high_value.columns if df_high_value[feature].isnull().sum() >= 29979*0.6]

In [ ]:
print(len(null_cols))
null_cols

In [ ]:
# the above features have too many missing values.  
# be best to drop them
df_high_value = df_high_value.drop(null_cols, axis=1)

In [ ]:
df_high_value.shape

#### Filter out features with less than 60% missing values

In [ ]:
#find features with less than 60% missing
null_col = [feature for feature in df_high_value.columns if df_high_value[feature].isnull().sum() > 0]

In [ ]:
print(len(null_col))
null_col

#### filter out date features

In [ ]:
#filter date cols
date_null_col = [feature for feature in null_col if 'date' in feature]

In [ ]:
date_null_col

#### Impute missing values

In [ ]:
#impute the date cols
for feature in date_null_col:
    df_high_value[feature].fillna(df_high_value[feature].mode()[0], inplace=True)

In [ ]:
#non date cols in null features
null_col_ = [feature for feature in null_col if 'date' not in feature]
null_col_

In [ ]:
#impute the non date cols in null_col_ with median of feature
for feature in null_col_:
    df_high_value[feature].fillna((df_high_value[feature].median()), inplace=True)

#### Final data check

In [ ]:
#check null % after dropping and imputing
round(df_high_value.isnull().sum()/len(df_high_value.index)*100, 2)

In [ ]:
#check shape after dropping
df_high_value.shape

In [ ]:
df_high_value.head(3)

In [ ]:
#get date cols
date_cols = [feature for feature in df_high_value.columns if 'date' in feature]
date_cols

#### Extract day from date cols

In [ ]:
#get day from date
for feature in df_high_value[date_cols]:
    df_high_value[feature] = pd.to_datetime(df_high_value[feature]).dt.day

In [ ]:
df_high_value.head(3)

#### Drop features with no predictive power ie. ONE unique value

In [ ]:
#if one unique value in feature drop it
single_value = [feature for feature in df_high_value.columns if df_high_value[feature].nunique() == 1]
print(len(single_value))
single_value

In [ ]:
#drop above cols
df_high_value = df_high_value.drop(single_value, axis=1)

In [ ]:
df_high_value.shape

In [ ]:
df_high_value.head(3)

# EDA

In [ ]:
#churn count
sns.countplot(x='churn', data=df_high_value, hue='churn')

In [ ]:
#print box plots for all features vs. churn
for f in numerical.columns:
    sns.boxplot(x=df_high_value['churn'], y=f, data=df_high_value)
    plt.show()

In [ ]:
plotScatterMatrix(df_high_value, 20, 10)

### Outlier treatment

In [ ]:
#Check outliers in the data 
df_high_value.describe(percentiles=[0.01, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99])

In [ ]:
#filter numerical columns 
numerical = df_high_value.select_dtypes(include=['int64', 'float'])

In [ ]:
#filter out churn + mobile number
numerical.drop(columns=['churn', 'mobile_number'], axis=1, inplace=True)
numerical.head(3)

In [ ]:
Q1 = numerical.quantile(0.01)
Q3 = numerical.quantile(0.99)
IQR = Q3 - Q1
print(IQR)

In [ ]:
#IQR range
df_high_value = df_high_value[~(((df_high_value < (Q1 - 1.5 * IQR)) | (df_high_value > (Q3 + 1.5 * IQR))).any(axis=1))]
print(df_high_value.shape)

### Split data into X and Y

In [ ]:
#split into x, y
X = df_high_value.drop(['churn', 'mobile_number', 'total_usage'], axis=1)
X.head(3)

In [ ]:
y = df_high_value['churn']

In [ ]:
y.head()

#### Scale and split the data

In [ ]:
scaler = StandardScaler()

In [ ]:
X_scaled = scaler.fit_transform(X)

In [ ]:
#tts
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, train_size=0.75, random_state=42)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

### Logistic Regression with RFE

In [ ]:
lr = LogisticRegression(class_weight='balanced')

In [ ]:
rfe = RFE(lr, 15)
rfe = rfe.fit(X_train, y_train)

In [ ]:
rfe.support_

In [ ]:
rfe_features = list(X.columns[rfe.support_])
print("Top 15 features ", rfe_features)

In [ ]:
X_train_rfe = pd.DataFrame(data=X_train).iloc[:, rfe.support_]
y_train_rfe = y_train

In [ ]:
lr = LogisticRegression(random_state=42)
lr.fit(X_train_rfe, y_train_rfe)

In [ ]:
#predict on train set
y_pred_train = lr.predict(X_train_rfe)
y_pred_train[:10]

In [ ]:
X_test_rfe = pd.DataFrame(data=X_test).iloc[:, rfe.support_]
y_test_rfe = y_test

In [ ]:
#predict on the test set
y_pred_test = lr.predict(X_test_rfe)
y_pred_test[:10]

In [ ]:
#accuracy
print('Accuracy-train:',metrics.accuracy_score(y_train, y_pred_train))
print('Accuracy-test:',metrics.accuracy_score(y_test, y_pred_test))

In [ ]:
#classification report
print(classification_report(y_test, y_pred_test))

In [ ]:
from sklearn.metrics import confusion_matrix
#confusion matrix on train data
confusion_matrix_train = metrics.confusion_matrix(y_train, y_pred_train)
print(confusion_matrix_train)

In [ ]:
#confusion matrix on test data
confusion_matrix_test = metrics.confusion_matrix(y_test, y_pred_test)
print(confusion_matrix_test)

In [ ]:
TP = confusion_matrix_train[1,1] # true positive 
TN = confusion_matrix_train[0,0] # true negatives
FP = confusion_matrix_train[0,1] # false positives
FN = confusion_matrix_train[1,0] # false negatives

In [ ]:
# sensitivity of our logistic regression model
TP / float(TP+FN)

In [ ]:
# specificity
TN / float(TN+FP)

#### ROC + AUC

In [ ]:
def draw_roc( actual, probs ):
    fpr, tpr, thresholds = metrics.roc_curve( actual, probs,
                                              drop_intermediate = False )
    auc_score = metrics.roc_auc_score( actual, probs )
    plt.figure(figsize=(5, 5))
    plt.plot( fpr, tpr, label='ROC curve (area = %0.2f)' % auc_score )
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate or [1 - True Negative Rate]')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

    return None

In [ ]:
draw_roc(y_train, y_pred_train)

In [ ]:
draw_roc(y_test, y_pred_test)

### Run PCA

In [ ]:
#create object
pca = PCA(random_state=42)

In [ ]:
#fit data
pca.fit(X_train)

In [ ]:
#transform train set
X_train_pca = pca.fit_transform(X_train)

In [ ]:
#transform test set
X_test_pca = pca.transform(X_test)

In [ ]:
#pca components
pca.components_

In [ ]:
#explained variance ratio
pca.explained_variance_ratio_

In [ ]:
#cumulative sum
var_cumu = np.cumsum(pca.explained_variance_ratio_)
var_cumu

### Logisitic regression with PCA

In [ ]:
#instantiate
lr_pca = LogisticRegression(C=1e9, class_weight='balanced')

In [ ]:
#fit
lr_pca.fit(X_train_pca, y_train)

In [ ]:
#predict
y_pred_test = lr_pca.predict(X_test_pca)

In [ ]:
#accuracy
print('Accuracy on the test dataset:',metrics.accuracy_score(y_test, y_pred_test))

In [ ]:
#confusion matrix
confusion_matrix_test = metrics.confusion_matrix(y_test, y_pred_test)
print(confusion_matrix_test)

### Scree Plot

In [ ]:
fig = plt.figure(figsize=[6,6])
plt.vlines(x=56, ymax=1, ymin=0, colors='c', linestyles='--')
plt.hlines(y=0.90, xmax=120, xmin=0, colors='g', linestyles='--')
plt.plot(var_cumu)
plt.xlabel('# Features')
plt.ylabel('Cumulative Variance Explained')
plt.show()

In [ ]:
#plot explained variance
plt.bar(range(1,len(pca.explained_variance_ratio_)+1), pca.explained_variance_ratio_)

### Final PCA with 56 features

In [ ]:
#object
pca_final = IncrementalPCA(n_components=56)

In [ ]:
#fit_transform train set
X_train_pca_56 = pca_final.fit_transform(X_train)

In [ ]:
#transform test set
X_test_pca_56 = pca_final.transform(X_test)

In [ ]:
X_train_pca_56.shape, X_test_pca_56.shape

#### Logistic model with PCA =56

In [ ]:
#instantiate
lr_pca56 = LogisticRegression(C=1e9, class_weight='balanced')

In [ ]:
#fit the data
lr_pca56.fit(X_train_pca_56, y_train)

In [ ]:
#predict
y_pred56 = lr_pca56.predict(X_test_pca_56)

In [ ]:
#accuracy
print('Accuracy on the test dataset:',metrics.accuracy_score(y_test, y_pred56))

In [ ]:
#confusion
confusion_matrix_test = metrics.confusion_matrix(y_test, y_pred56)
print(confusion_matrix_test)

In [ ]:
#correlation matrix
corrmat = np.corrcoef(X_train_pca_56.transpose())

In [ ]:
corrmat.shape

In [ ]:
#heatmap
plt.figure(figsize=[20,18])
sns.heatmap(corrmat, annot=True)

In [ ]:
def draw_roc( actual, probs ):
    fpr, tpr, thresholds = metrics.roc_curve( actual, probs,
                                              drop_intermediate = False )
    auc_score = metrics.roc_auc_score( actual, probs )
    plt.figure(figsize=(5, 5))
    plt.plot( fpr, tpr, label='ROC curve (area = %0.2f)' % auc_score )
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate or [1 - True Negative Rate]')
    plt.ylabel('True Positive Rate')
    plt.title('ROC')
    plt.legend(loc="lower right")
    plt.show()

    return None

draw_roc(y_test, y_pred56)

## Decision Tree Classifier

In [ ]:
#instantiate an object 
#deal with class imbalance
dt1 = DecisionTreeClassifier(class_weight='balanced', 
                             max_depth = 6)

#fit the model 
dt1.fit(X_train_pca_56, y_train)

In [ ]:
#predict on test set 
dt1_pred = dt1.predict(X_test_pca_56)

In [ ]:
#classification report
print(classification_report(y_test, dt1_pred))

In [ ]:
#confusion matrix
print(confusion_matrix(y_test, dt1_pred))

In [ ]:
#accuracy score
print(accuracy_score(y_test, dt1_pred))

### Grid Search to find optimal hyperparameters

#### tuning max_depth

In [ ]:
#grid search for optimal max_depth 
n_folds = 6
parameters = {'max_depth': range(1,40)}

#instantiate an object
dt2 = DecisionTreeClassifier(criterion = 'gini', random_state=42)

#fit on tree
tree = GridSearchCV(dt2, parameters, cv=n_folds, scoring='accuracy',
                   return_train_score=True, n_jobs=-1)

tree.fit(X_train_pca, y_train)

In [ ]:
#scores
score = tree.cv_results_
pd.DataFrame(score).head()

In [ ]:
#plot train vs. test accuracy
plt.figure()
plt.plot(score["param_max_depth"], 
         score["mean_train_score"], 
         label="training accuracy")
plt.plot(score["param_max_depth"], 
         score["mean_test_score"], 
         label="test accuracy")
plt.xlabel("max_depth")
plt.ylabel("Accuracy")
plt.legend()
plt.show()
#max_depth =3 seems to be the right fit

#### tuning min samples leaf

In [ ]:
#optimal value of minimum sample leaf
n_folds = 6

parameters = {'min_samples_leaf': range(5,200,25)}

dt3 = DecisionTreeClassifier(criterion = 'gini', random_state=42)

#fit on tree
tree2 = GridSearchCV(dt3, parameters, cv=n_folds, scoring='accuracy',
                   return_train_score=True, n_jobs=-1)

tree2.fit(X_train_pca, y_train)

In [ ]:
#scores
score = tree2.cv_results_
pd.DataFrame(score).head()

In [ ]:
#plot train vs. test accuracy
plt.figure()
plt.plot(score["param_min_samples_leaf"], 
         score["mean_train_score"], 
         label="training accuracy")
plt.plot(score["param_min_samples_leaf"], 
         score["mean_test_score"], 
         label="test accuracy")
plt.xlabel("max_depth")
plt.ylabel("Accuracy")
plt.legend()
plt.show()
#min_samples_split = 26 seems to be the choice

#### tuning min_samples_split

In [ ]:
#tune param_min_samples_split
n_folds = 6

parameters = {'min_samples_split' : range(5, 200, 20)}

dt4 = DecisionTreeClassifier(criterion = 'gini', random_state=42)

tree3 = GridSearchCV(dt4, parameters, cv=n_folds, scoring='accuracy',
                    return_train_score=True, n_jobs=-1)
tree3.fit(X_train_pca, y_train)

In [ ]:
#scores
score = tree3.cv_results_
pd.DataFrame(score).head()

In [ ]:
#plot train vs. test accuracy
plt.figure()
plt.plot(score["param_min_samples_split"], 
         score["mean_train_score"], 
         label="training accuracy")
plt.plot(score["param_min_samples_split"], 
         score["mean_test_score"], 
         label="test accuracy")
plt.xlabel("max_depth")
plt.ylabel("Accuracy")
plt.legend()
plt.show()
#param_min_samples_split = 40 is optimal

#### create parameter grid

In [ ]:
# Create parameter grid 
param_grid = {
    'max_depth': range(5, 15, 5),
    'min_samples_leaf': range(25, 175, 50),
    'min_samples_split': range(50, 150, 50),
    'criterion': ["entropy", "gini"]
}

n_folds = 6

# Instantiate grid search
tree4 = DecisionTreeClassifier()
grid_search = GridSearchCV(estimator = tree4, param_grid = param_grid, 
                          cv = n_folds, n_jobs = -1)

# Fit the grid search 
grid_search.fit(X_train_pca, y_train)

In [ ]:
cv_results = pd.DataFrame(grid_search.cv_results_)
cv_results
print('Best Accuracy:', grid_search.best_score_)
print(grid_search.best_estimator_)

#### Final Decision Tree with optimal hyperparameters

In [ ]:
#final model with optimal hyperparameters 
dt_final = DecisionTreeClassifier(criterion='entropy', max_depth=5,
                                 min_samples_leaf=25,
                                 min_samples_split=50,
                                 random_state=42)
dt_final.fit(X_train_pca, y_train)

In [ ]:
#accuracy on test data
print('Accuracy score:', dt_final.score(X_test_pca, y_test))

## Random Forest

In [ ]:
#object
rf1 = RandomForestClassifier()
rf1.fit(X_train_pca, y_train)

#predict
rf_pred_test = rf1.predict(X_test_pca)

In [ ]:
#accuracy
print('Random Forest with default hyperparameters:', 
      metrics.accuracy_score(y_test, rf_pred_test))

In [ ]:
#classification report 
print(classification_report(y_test, rf_pred_test))

In [ ]:
#confusion matrix
print(confusion_matrix(y_test, rf_pred_test))

#### Grid Search to find optimal hyperparameters 

#### Tuning max_depth

In [ ]:
#folds 
n_folds = 6

#max_depth 
parameters = {'max_depth' : range(2, 22, 6)}

#instantiate
rf2 = RandomForestClassifier()

#fit 
gc_rf = GridSearchCV(rf2, parameters, cv=n_folds, scoring='accuracy',
                    return_train_score=True, n_jobs=-1)

gc_rf.fit(X_train_pca, y_train)

In [ ]:
#scores
scores = gc_rf.cv_results_
pd.DataFrame(scores).head()

In [ ]:
#plot train vs. test accuracy
plt.figure()
plt.plot(scores["param_max_depth"], 
         scores["mean_train_score"], 
         label="training accuracy")
plt.plot(scores["param_max_depth"], 
         scores["mean_test_score"], 
         label="test accuracy")
plt.xlabel("max_depth")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

#### Tuning n_estimators

In [ ]:
#folds 
n_folds = 6

#max_depth 
parameters = {'n_estimators' : range(100, 1600, 500)}

#instantiate
rf3 = RandomForestClassifier(max_depth=8)

#fit 
rf3 = GridSearchCV(rf3, parameters, cv=n_folds, scoring='accuracy',
                    return_train_score=True, n_jobs=-1)

rf3.fit(X_train_pca, y_train)

In [ ]:
#scores
scores = rf3.cv_results_
pd.DataFrame(scores).head()

In [ ]:
#plot train vs. test accuracy
plt.figure()
plt.plot(scores["param_n_estimators"], 
         scores["mean_train_score"], 
         label="training accuracy")
plt.plot(scores["param_n_estimators"], 
         scores["mean_test_score"], 
         label="test accuracy")
plt.xlabel("max_depth")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

#### Tuning min_samples_leaf

In [ ]:
#folds 
n_folds = 6

#max_depth 
parameters = {'min_samples_leaf' : range(50, 400, 100)}

#instantiate
rf4 = RandomForestClassifier()

#fit 
rf4 = GridSearchCV(rf4, parameters, cv=n_folds, scoring='accuracy',
                    return_train_score=True, n_jobs=-1)

rf4.fit(X_train_pca, y_train)

In [ ]:
#scores
scores = rf4.cv_results_
pd.DataFrame(scores).head()

In [ ]:
#plot train vs. test accuracy
plt.figure()
plt.plot(scores["param_min_samples_leaf"], 
         scores["mean_train_score"], 
         label="training accuracy")
plt.plot(scores["param_min_samples_leaf"], 
         scores["mean_test_score"], 
         label="test accuracy")
plt.xlabel("max_depth")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

#### Tuning min_samples_split

In [ ]:
#folds 
n_folds = 6

#max_depth 
parameters = {'min_samples_split' : range(100, 500, 30)}

#instantiate
rf5 = RandomForestClassifier()

#fit 
rf5 = GridSearchCV(rf5, parameters, cv=n_folds, scoring='accuracy',
                    return_train_score=True, n_jobs=-1)

rf5.fit(X_train_pca, y_train)

In [ ]:
#scores
scores = rf5.cv_results_
pd.DataFrame(scores).head()

In [ ]:
#plot train vs. test accuracy
plt.figure()
plt.plot(scores["param_min_samples_split"], 
         scores["mean_train_score"], 
         label="training accuracy")
plt.plot(scores["param_min_samples_split"], 
         scores["mean_test_score"], 
         label="test accuracy")
plt.xlabel("max_depth")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:
#parameter grid
param_grid = {
    'max_depth': [4,8,10],
    'min_samples_leaf': range(100, 300, 100),
    'min_samples_split': range(200, 500, 100),
    'n_estimators': [500,700], 
    'max_features': [10,20,25]
}

rf = RandomForestClassifier()

gs_rf = GridSearchCV(estimator = rf, param_grid = param_grid, n_jobs=-1, verbose=1)

gs_rf.fit(X_train_pca, y_train)

In [ ]:
print('Accuracy Score:', gs_rf.best_score_)
print('Best parameters:', gs_rf.best_params_)

### Random forest with most important features and best params

# I was unable to run grid search above as my laptop is very slow. Hence the hyperparameters below are missing. The code is correct and will work if hyperparamters are entered as per grid search. My apologies but my laptop is from 2014. 

In [ ]:
#object
rf_final = RandomForestClassifier(max_depth=,
                                 min_samples_leaf=,
                                 min_samples_split=,
                                 n_estimators=,
                                 max_features=)

#fit
rf_final.fit(X_train_pca, y_train)

#predict
rf_y_pred = rf_final.predict(X_test_pca)

In [ ]:
#accuracy 
print('Accuracy:', metrics.accuracy_score(y_test, rf_y_pred))

In [ ]:
#classification report 
print(classification_report(y_test, rf_y_pred))

In [ ]:
#confusion matrix
print(confusion_matrix(y_test, rf_y_pred))

Strategies

Month 8 features are the most important features to predict churn

If customer behaviour becomes erratic in month 8, they are likely to churn

In order to manage churn, telecom operators need to get aggressive in previous months leading upto churn with
Discounts and attractive offers, free talk time or data could prove to be beneficial
Improve customer service especially in previous months to change customers mind

Model Selection

Random forest Classifier model works best to predict telecom churn based on accuracy score and classification report. Since we are more soncerned with tagging the churners vs. the non churners correctly, random forest does well for our problem.